## Assignment 2
# Group 13
Mathieu Mailhot - Isabel Lougheed - Frank-Lucas Pantazis

In [89]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv
import os
import re

# Text Preprocessing
import nltk
from sklearn.feature_extraction import text
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize
from nltk import pos_tag


nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

## Checkout link: https://www.analyticsvidhya.com/blog/2021/11/a-guide-to-building-an-end-to-end-multiclass-text-classification-model/

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\frank\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\frank\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\frank\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\frank\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\frank\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [90]:
# Hyperparameter
folds = 30 # between 5 and 10 # best value at the moment when folds = 30

# Loading Training data
df_train = pd.read_csv('train.csv', encoding='utf-8', encoding_errors='ignore') # errors were not pertinent characters
df_train["subreddit"] = df_train["subreddit"].map({"Boston": 0, "Canberra": 1,"Geneva":2,"Ottawa":3})

y = df_train["subreddit"]
X = df_train["body"]

print(X)

# Loading Test Data
df_test = pd.read_csv('test.csv', encoding='utf-8', encoding_errors='ignore') # errors were not pertinent characters
 # Not what we should do with the ID

0       I had to put in a drain well/french drain, and...
1       I've worked with James from Prova accountants ...
2       https://lebonmelange.com.au/ is a Gungahlin ca...
3       What I love about Canberra are the town planni...
4       Canberra has a bigger issue with strata. Rates...
                              ...                        
1395    Take the train to La pleine and walk to joncti...
1396    IIL alumni here!\n\nGeneva private schools are...
1397    I'm really sorry to hear about your bad experi...
1398    They quite easy to handle. Typically, they don...
1399    **Specialization is authentic.** You don?t go ...
Name: body, Length: 1400, dtype: object


<h2>Text Preprocessing</h2>

<h3>Helper Functions</h3>

In [185]:
lemmatizer = WordNetLemmatizer()

def lemmatize_tokenizer(subreddit_post, stop_word_catalogue=None):
    subreddit_post = subreddit_post.lower()
    words = word_tokenize(subreddit_post)
    # WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV
    
    lemmatized_words = []
    
    word_Lemmatized = WordNetLemmatizer()
    for word, tag in pos_tag(words):
        # Below condition is to check for Stop words and consider only alphabets
        condition = True
        if (stop_word_catalogue!=None):
            condition = word not in stop_word_catalogue
        if(condition):
            if word.isalpha():
                lemmatized_word = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
                #print(word,lemmatized_word)
                lemmatized_words.append(lemmatized_word)                   
    return lemmatized_words

def preprocessing_data(X, stop_word_catalogue):
    lemmatized_data = []
    for i in range(len(X)):
        post = X[i]
        lemmatized_post = lemmatize_tokenizer(post,stop_word_catalogue)
        lemmatized_data.append(" ".join(lemmatized_post))
    return lemmatized_data

def feature_extraction(X,ngram_range,min_df,caption=""):
    vectorizer = CountVectorizer(ngram_range=ngram_range,min_df=min_df)
    X_disp = vectorizer.fit_transform(X)
    feature_names = vectorizer.get_feature_names_out()
    print(caption,"features:", len(feature_names))
    return feature_names

<h3> Preprocessing </h3>

In [ ]:
# Stop Word Catalogues
nltk_stop_words = stopwords.words('english')
sklean_stop_words = list(text.ENGLISH_STOP_WORDS)
stop_words_arr = list(text.ENGLISH_STOP_WORDS.union(nltk_stop_words))

# Trainning Data
pp_df_train_1 = preprocessing_data(X,[])
pp_df_train_2 = preprocessing_data(X,nltk_stop_words)
pp_df_train_3 = preprocessing_data(X,sklean_stop_words)
pp_df_train_4 = preprocessing_data(X,stop_words_arr)

pp_df_train = [X,pp_df_train_1,pp_df_train_2,pp_df_train_3,pp_df_train_4]

# Testing Data
lemmatized_df_test = []
for i in range(len(df_test["body"])):
    post = df_test["body"][i]
    lemmatized_post_test = lemmatize_tokenizer(post,stop_words_arr)
    lemmatized_df_test.append(" ".join(lemmatized_post_test))

In [190]:
# Lemmatized data
for i in range(len(pp_df_train)): 
    elem = pp_df_train[i]
    if (i==0):
        feature_extraction(elem,(1,1),2,f"Unpreprocessed")
    else:
        feature_extraction(elem,(1,1),2,f"Preprocessed {i}")



Unpreprocessed features: 5631
Preprocessed 1 features: 4385
Preprocessed 2 features: 4272
Preprocessed 3 features: 4175
Preprocessed 4 features: 4156


<h3>Important words<h3>

In [319]:
meaningfull_words_list ={ "features", "place", "city", "live", "area", "home", "local", "park", "house", 
    "street", "town", "building", "centre", "apartment", "housing", "space", "private", 
    "neighbour", "quality", "location", "property", "view", "garden", "living", "flat", 
    "landlord", "suburb", "rental", "neighborhood", "community", "state", 
    "public", "bus", "car", "drive", "train", "tax", "cost", "rent", "price", 
    "job", "money", "income", "family", "food", "health", "education", "shopping", 
    "restaurant", "work", "school", "university", "college", "traffic", "parking", 
    "safety", "business", "market", "weather", "snow", "summer", "lake", "mountain", 
    "river", "suburb", "downtown", "north", "south", "east", "west", "coast", "council", 
    "state", "village", "region", "distance", "commute",
    "belconnen", "civic", "kingston", "fyshwick", "gungahlin", "woden", "ontario", "neighborhood", 
    "neighbourhood", "rideau", "servette", "belmont", "bilingual", "capital", "chinese", "clinic", 
    "dickson", "east", "europe", "farm", "grocery", "immigration", "jonction", "lady", "loud", "mall", 
    "metrolinx", "montreux", "officer", "paquis", "port", "province", "rain", "reading", "resolve", 
    "rush", "sector", "se", "separate", "toronto", "village", "wind", "vancouver", "vieux", "windsor", 
    "bell", "bakery", "bar", "boston", "canton", "centre", "city", "club", "commune", "downtown", "drive", 
    "family", "flat", "food", "france", "gardens", "glen", "grocery", "home", "housing", "indirect", "island", 
    "land", "location", "market", "metro", "neighbour", "neighborhood", "north", "paris", "place", "pool", 
    "province", "queensland", "rain", "river", "road", "shop", "shopping", "suburb", "village", "walk", "water", 
    "west", "woden",
    "brisbane", "brookline", "byward", "carouge", "canal", "centre", "china", "classic", "climate", 
    "commuter", "cornavin", "davis", "dealer", "dickson", "downtown", "expat", "fenway", "ferney", 
    "garage", "gentleman", "grocery", "hassle", "hospital", "indoor", "kanata", "kitchen", "lansdowne", 
    "london", "malden", "majority", "master", "metrolinx", "mont", "montreal", "nsw", "ont", "orleans", 
    "parkway", "queanbeyan", "regina", "subway", "subaru", "sydney", "toronto", "vancouver", "ville", 
    "wellington", "western", "windsor", "braddon", "canberrans", "carleton", "centretown", "chestnut", 
    "citizenship", "commercial", "comparison", "competitive", "coworkers", "cultural", "cumming", 
    "culture", "deep", "detour", "discover", "divert", "donut", "dossier", "employ", "employment", 
    "enforce", "entertainment", "exchange", "famous", "frontaliers", "garbage", "gate", "gender", 
    "glebe", "growth", "hassle", "honest", "huntsville", "identify", "integrate", "interior", "introduce", 
    "itinerary", "jacket", "july", "kanata", "lifestyle", "loop", "lunch", "mayor", "membership", 
    "metro", "mexican", "midnight", "minority", "monday", "monster", "natural", "newbury", "operation", 
    "orleans", "outdoor", "park", "peux", "phd", "photograph", "plan", "plow", "post", "precinct", "press", 
    "promotion", "pump", "propose", "quebec", "realistic", "recycle", "reference", "reflect", "refund", 
    "remind", "repeat", "represent", "reservation", "retail", "rip", "rise", "roller",
    "acton", "advertise", "alewife", "arlington", "avenue", "barrier", "billion", "bishop", "boat", 
    "boston", "braddon", "brockton", "bro", "cambridge", "canal", "canton", "cent", "chateau", "channel", 
    "cher", "chicago", "clearing", "cliff", "columbus", "cornavin", "countryside", "darling", "downtown", 
    "eaux", "embassy", "esplanade", "fais", "faith", "fen", "fence", "flood", "footpath", "franais", "frontire", 
    "gatineau", "geneva", "gva", "hayden", "hudson", "institute", "jardin", "jfk", "kenmore", "lyon", "london", 
    "louvre", "malden", "manchester", "meadow", "melbourne", "metropolitan", "meyrin", "miami", "montreal", 
    "nyc", "ontario", "ottawa", "paris", "park", "perception", "quincy", "regina", "residential", "riverside", 
    "rome", "saconnex", "sbb", "sherbrooke", "shore", "shut", "slide", "south", "spain", "sydney", "temple", "toronto", 
    "vancouver", "vives", "washington", "west", "wellington", "zoo",
    "danvers", "dcembre", "genevan", "gex", "halifax", "hilarious", "hindmarsh", "hong", "idp", 
    "jamison", "jan", "jean", "kendall", "kippax", "kong", "lancy", "logan", "meadow", "melrose", 
    "merivale", "mgb", "mawson", "monitor", "murray", "nyon", "pacific", "parc", "parliament", 
    "philipp", "plaza", "providence", "rochester", "rural", "saint", "somerset", "somerville", 
    "southie", "southside", "suburban", "sunset", "survey", "territory", "tugger", "vanier", 
    "versoix", "virginia", "warren", "wednesday", "windy", "whatsapp", "toyota",
    "barrhaven", "basel", "bathurst", "bernie", "bostonian", "boulangerie", "boulevard", 
    "bowral", "brighton", "burlington", "calwell", "charlestown", "charnwood", "chicago", 
    "claridge", "clover", "coastal", "collaboration", "colonization", "colony", "commune", 
    "community", "coordinate", "council", "corriidor", "culturally", "darwin", "decide", 
    "dublin", "dunsborough", "escalator", "florida", "france", "fremantle", "goulburn", 
    "greatly", "grouse", "gta", "heavily", "hilton", "honk", "institute", "intensive", "interstate", 
    "ireland", "jamaica", "japan", "joint", "jura", "katoomba", "kenya", "kilometre", "kitchen", 
    "krispy", "liverpool", "luxembourg", "manuka", "melbourne", "mines", "motorbike", "montreal", 
    "ncc", "newly", "ny", "oak", "obama", "october", "olympic", "paradise", "parliament", 
    "parkdale", "parkour", "perth", "pioneer", "pound", "port", "postcard", "prescription",
    "puplinge", "qld", "quebec", "riverside", "roadway", "roxbury", "salem", "seattle", "sandy", 
    "seaport", "spain", "spanish", "stoneham", "suburbs", "sydney", "tavern", "territoire", "theater", 
    "tuggeranong", "vancouver", "waltham", "wanniassa", "ward", "warehouse", "wellington", "westboro", 
    "weston", "wetland", "wholesale", "wilson", "winchester", "winery", "yarralumla"}


<h3>Stop Word Custom (TO MODIFY)</h3>

In [263]:
# Creating new stopword list
vectorizer = CountVectorizer(stop_words='english',ngram_range=(1, 1),min_df=1)
X_stop = vectorizer.fit_transform(pp_df_train_1)
feature_names = vectorizer.get_feature_names_out()

# Checkout: https://towardsdatascience.com/text-pre-processing-stop-words-removal-using-different-libraries-f20bac19929a/
Boston_counts = X_stop[y == 0].sum(axis=0).A1 # Sum occurrences for class 'Boston'
Canberra_counts = X_stop[y == 1].sum(axis=0).A1 # Sum occurrences for class 'Canberra'
Geneva_counts = X_stop[y == 2].sum(axis=0).A1 # Sum occurrences for class 'Geneva'
Ottawa_counts = X_stop[y == 3].sum(axis=0).A1 # Sum occurrences for class 'Ottawa'

header = ["features","Boston","Canberra","Geneva","Ottawa","Count"]
custom = []
for i in range(len(feature_names)):
    k = [Boston_counts[i].item(),Canberra_counts[i].item(),Geneva_counts[i].item(),Ottawa_counts[i].item()]
    a = sum([Boston_counts[i].item(),Canberra_counts[i].item(),Geneva_counts[i].item(),Ottawa_counts[i].item()])
    crit  = abs(min(k) - max(k))/ a
    if (crit<=0.1 and (crit*a)/max(k)<=0.20):
        custom.append(feature_names[i])
print(custom)
print("Initial Stop Word Size:",len(text.ENGLISH_STOP_WORDS))
stop_words_custom = list(text.ENGLISH_STOP_WORDS.union(custom))
print("Custom Stop Word Size:",len(stop_words_custom))

['ad', 'aid', 'bias', 'coach', 'come', 'correct', 'director', 'effective', 'engagement', 'flip', 'good', 'imagine', 'invest', 'new', 'pack', 'perfect', 'rid', 'rink', 'significantly', 'situation', 'star', 'strategy', 'suddenly', 'sure', 'thing', 'think', 'trend', 'utility']
Initial Stop Word Size: 318
Custom Stop Word Size: 346


<h3>Train</h3>

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import KFold
from sklearn.decomposition import NMF
# Observations
# - bigram -> worse performance
# - sublinear_tf -> seems to improve accuracy
# - decreasing max_features -> seems to decrease accuracy (feature reduction)

# TODO
# - Create custom stop word list since default one might not be suited for our case according to documentation: https://scikit-learn.org/stable/modules/feature_extraction.html#stop-words
# - explore different ways to extract features from text data

# Instantiate Vectorizer
tfidf_uni = TfidfVectorizer(ngram_range=(1, 1), sublinear_tf=True, stop_words=stop_words_custom)
tfidf_uni_bi = TfidfVectorizer(max_features=5000, ngram_range=(1, 2),sublinear_tf=True, stop_words=stop_words_custom)
tfidf_bi = TfidfVectorizer(ngram_range=(2, 2), sublinear_tf=True,min_df=2, stop_words=stop_words_custom)
naiveBayes_uni = CountVectorizer(max_features=3000, ngram_range=(1, 1), stop_words=stop_words_custom)


# Fit Vectorizer from data
X_uni = tfidf_uni.fit_transform(lemmatized_df_train).toarray()
X_uni_bi = tfidf_uni_bi.fit_transform(lemmatized_df_train).toarray()
X_bi = tfidf_bi.fit_transform(X).toarray()
X_naive_bayes = naiveBayes_uni.fit_transform(X).toarray()



#X_test = tfidf_uni_bi.transform(lemmatized_df_test)


print("Unigram", "(size:",str(len(tfidf_uni.get_feature_names_out()))+")")
print(tfidf_uni.get_feature_names_out())
#print(tfidf_uni.get_feature_names_out())
print("Unigram & Bigram", "(size:",str(len(tfidf_uni_bi.get_feature_names_out()))+")")
#print(tfidf_uni_bi.get_feature_names_out())
print("Bigram", "(size:",str(len(tfidf_bi.get_feature_names_out()))+")")
#print(tfidf_bi.get_feature_names_out())

# LOOK INTO NMF AND WHAT IT CAN DO TO HELP US
#nmf = NMF(100).fit(X_uni)

#for topic_idx, topic in enumerate(nmf.components_):
#    top_features_ind = topic.argsort()[-10:]
#    top_features = tfidf_uni.get_feature_names_out()[top_features_ind]
#    print(top_features)
    
# To get a better idea of the extracted features
with open("features.csv", mode='w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # Write a header (optional, if you want)
    writer.writerow(["Feature Name"])
    # Write the features from the array
    for feature in tfidf_uni.get_feature_names_out():
        writer.writerow([feature])

with open("featuresNaiveBayes.csv", mode='w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # Write a header (optional, if you want)
    writer.writerow(["Feature Name"])
    # Write the features from the array
    for feature in naiveBayes_uni.get_feature_names_out():
        writer.writerow([feature])

Unigram (size: 12835)
['00' '000' '0001' ... 'zucchini' 'zurich' 'zxwr7mvro1z3kabb']
Unigram & Bigram (size: 5000)
Bigram (size: 2246)


In [ ]:
#tfidf_uni_arr = []
#X_uni_arr = []
#for i in range(len(pp_df_train)):
#    tfidf = TfidfVectorizer(ngram_range=(1, 1), min_df=2,sublinear_tf=True, stop_words=stop_words_custom)
#    X_uni_arr.append(np.power(tfidf.fit_transform(pp_df_train[i]).toarray(), 2) )
#    tfidf_uni_arr.append(tfidf)
#    print("Unigram", "(size:",str(len(tfidf.get_feature_names_out()))+")")

tfidf_primary = TfidfVectorizer(vocabulary=meaningfull_words_list)
X_uni_primary = tfidf_primary.fit(pp_df_train[0])
tfidf_secondary = TfidfVectorizer(ngram_range=(1, 1),min_df=2,sublinear_tf=True, stop_words=stop_words_custom)
X_uni_secondary = tfidf_secondary.fit(pp_df_train[0])
print("Unigram", "(size:",str(len(tfidf_primary.get_feature_names_out()))+")")
print("Unigram", "(size:",str(len(tfidf_secondary.get_feature_names_out()))+")")
#tfidf_uni_bi_arr = []
#X_uni_bi_arr = []
#for i in range(len(pp_df_train)):
#    tfidf = TfidfVectorizer(ngram_range=(1, 2), min_df=2, sublinear_tf=True,stop_words="english")
#    X_uni_bi_arr.append( tfidf.fit_transform(pp_df_train[i]).toarray())
#    tfidf_uni_bi_arr.append(tfidf)
#    print("Unigram & Bigram", "(size:",str(len(tfidf.get_feature_names_out()))+")")

['now', 'cant', 'anyway', 'inc', 'ie', 'am', 'wherever', 'hers', 'first', 'during', 'your', 'our', 'of', 'up', 'found', 'flip', 'once', 'aid', 'be', 'as', 'hundred', 'else', 'whoever', 'and', 'out', 'some', 'toward', 'anyone', 'ten', 'herein', 'hence', 'was', 'above', 'himself', 'his', 'together', 'again', 'twelve', 'might', 'behind', 'towards', 'although', 'nobody', 'in', 'six', 'yourselves', 'alone', 'everything', 'former', 'with', 'i', 'see', 'everywhere', 'moreover', 'perfect', 'he', 'nor', 'whole', 'pack', 'what', 'often', 'anything', 'serious', 'otherwise', 'already', 'those', 'do', 'you', 'further', 'around', 'being', 'name', 'eg', 'besides', 'same', 'trend', 'hasnt', 'etc', 'too', 'a', 'sure', 'nevertheless', 'de', 'co', 'less', 'ever', 'seemed', 'well', 'ad', 'sincere', 'whereupon', 'per', 'itself', 'bias', 'becoming', 'latter', 'several', 'mine', 'though', 'fifty', 'more', 'side', 'due', 'both', 'give', 'afterwards', 'invest', 'few', 'thick', 'before', 'this', 'imagine', 'get

In [ ]:
# Draft of Feature Visualizer
# Maybe should put all of it in an excel and then display it?
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words='english',ngram_range=(1, 1),min_df=2)

X_disp = vectorizer.fit_transform(pp_df_train[4])
feature_names = vectorizer.get_feature_names_out()

Boston_counts = X_disp[y == 0].sum(axis=0).A1 # Sum occurrences for class 'Boston'
Canberra_counts = X_disp[y == 1].sum(axis=0).A1 # Sum occurrences for class 'Canberra'
Geneva_counts = X_disp[y == 2].sum(axis=0).A1 # Sum occurrences for class 'Geneva'
Ottawa_counts = X_disp[y == 3].sum(axis=0).A1 # Sum occurrences for class 'Ottawa'


header = ["features","Boston","Canberra","Geneva","Ottawa","Count"]
custom = []
table = []
for i in range(len(feature_names)):
    k = [Boston_counts[i].item(),Canberra_counts[i].item(),Geneva_counts[i].item(),Ottawa_counts[i].item()]
    a = sum([Boston_counts[i].item(),Canberra_counts[i].item(),Geneva_counts[i].item(),Ottawa_counts[i].item()])
    crit  = abs(min(k) - max(k))/ a

    #table.append([feature_names[i],Boston_counts[i].item(),Canberra_counts[i].item(),Geneva_counts[i].item(),Ottawa_counts[i].item(),a])
    if (crit>0.08 and (crit*a)/max(k)>0.2):
        table.append([feature_names[i],Boston_counts[i].item(),Canberra_counts[i].item(),Geneva_counts[i].item(),Ottawa_counts[i].item(),a])
    else:
        custom.append(feature_names[i])
print(custom)
table_sorted = sorted(table, key=lambda x: x[5], reverse=True)

if (True):
    with open("featureVisualiser5.csv", mode='w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        # Write a header (optional, if you want)
        writer.writerow(header)
        # Write the features from the array
        for row in table_sorted:
            writer.writerow(row)

['ad', 'add', 'advice', 'aid', 'allow', 'bias', 'coach', 'come', 'correct', 'director', 'discount', 'effective', 'engagement', 'example', 'explain', 'flip', 'good', 'hope', 'imagine', 'invest', 'need', 'new', 'open', 'pack', 'perfect', 'provide', 'return', 'rid', 'rink', 'say', 'significantly', 'situation', 'star', 'strategy', 'struggle', 'suddenly', 'sure', 'tell', 'thing', 'think', 'time', 'trend', 'utility', 'wear']


<h3>Helper Functions</h3>

In [369]:
import sklearn
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

# This function does all the tunning for each model
def hyperparamaterTunning(X,param, folds, model, verbose_val=1, Y=y):
    
    model_gridSearch = GridSearchCV(model, param_grid=param,cv=folds, verbose=verbose_val) # According to doc the data will be split the same way accross all calls

    model_best_clf = model_gridSearch.fit(X,Y)

    cv_results = model_gridSearch.cv_results_

    print()

    print(f"Best Parameters: {model_best_clf.best_params_}")
    try:

        best_index = model_gridSearch.best_index_

        score = []
        for fold in range(folds):
            score.append(model_gridSearch.cv_results_[f"split{fold}_test_score"][best_index].item())

        print(f"Cross-validation Accuracies: {score}")
    except:
        pass
    print(f"Mean Accuracy: {model_best_clf.best_score_:.4f}")

    return model_best_clf


def meaninfullWords(X,y,word_list):
    X_primary = []
    y_primary = []
    X_secondary= []
    y_secondary=[]
    for i in range(len(X)):
        appended  = False
        words  = X[i].split(" ")
        for word in words:
            if (word in word_list):
                X_primary.append(X[i])
                y_primary.append(y[i])
                appended = True
                break
        if (not appended):
            X_secondary.append(X[i])
            y_secondary.append(y[i])
    return (X_primary,y_primary,X_secondary,y_secondary)

<h2>Naive Bayes</h2>

In [38]:
class NaiveBayes:
    def __init__(self, x_all, y_all, feature_vectoriser):
        self.x_all = self.clean_text_data(x_all) # Make lists of strings
        self.y_all = y_all
        self.feature_vectoriser = feature_vectoriser

        self.folds_features_probability = 0 # array of dict
        self.folds_accuracy = 0
        self.avg_accuracy = 0

    
    def calc_probability(self, x, y): # Train/Fit # Mathieu
        # Create an empty dictionnary with the 3000 most common words for each subreddit.
        features_probability_boston = {word: 0 for word in self.feature_vectoriser.get_feature_names_out()}
        features_probability_canberra = {word: 0 for word in self.feature_vectoriser.get_feature_names_out()}
        features_probability_geneva = {word: 0 for word in self.feature_vectoriser.get_feature_names_out()}
        features_probability_ottawa = {word: 0 for word in self.feature_vectoriser.get_feature_names_out()}

        # Initialize the count for the total number of text from each subreddit
        count_boston = 0
        count_canberra = 0
        count_geneva = 0
        count_ottowa = 0

        # Add 1 to the word in the dictionnary when the word is present in the text
        for i in range(y.shape[0]):
            if y[i] == 0:
                count_boston += 1
                self.add_probability(features_probability_boston, x[i])
            if y[i] == 1:
                count_canberra += 1
                self.add_probability(features_probability_canberra, x[i])
            if y[i] == 2:
                count_geneva += 1
                self.add_probability(features_probability_geneva, x[i])
            else:
                count_ottowa += 1
                self.add_probability(features_probability_ottawa, x[i])

        # Add the total count of each city to a variable called "city_count" and the probability of each city in a variable called "city_probability" in each one of the dictionary
        features_probability_boston["city_count"] = count_boston
        features_probability_boston["city_probability"] = count_boston / (count_boston + count_canberra + count_geneva + count_ottowa)
        features_probability_canberra["city_count"] = count_canberra
        features_probability_canberra["city_probability"] = count_canberra / (count_boston + count_canberra + count_geneva + count_ottowa)
        features_probability_geneva["city_count"] = count_geneva
        features_probability_geneva["city_probability"] = count_geneva / (count_boston + count_canberra + count_geneva + count_ottowa)
        features_probability_ottawa["city_count"] = count_ottowa
        features_probability_ottawa["city_probability"] = count_ottowa / (count_boston + count_canberra + count_geneva + count_ottowa)
                
        return features_probability_boston, features_probability_canberra, features_probability_geneva, features_probability_ottawa # return all dictionaries

    def clean_text_data(self, x): # Helper function to make a list of lists of words 
        # Take text remove all capitalized letters, removed special characters and make an array of words.
        cleaned_data = [
            re.sub(r'[^a-z0-9\s]', '', text.lower()).split()
            for text in x
        ]
        print("This is the cleaned data", cleaned_data[0])
        return cleaned_data # return a list of lists of words (better to use lists for this since numpy is mostly for numerical values)
    
    def add_probability(self, city_dict, x): # Helper function to update probabilities given a dict and a list of words
        for word in set(x): # Creates a set from words(unique elements)
            if word in city_dict:
                city_dict[word] += 1
                
    
    def predict(self, features_probability_boston, features_probability_canberra, features_probability_geneva, features_probability_ottawa, x_i): # (Is x_i in formula equal to 1?)
        # Initialize probabilities for each subreddit
        prob_boston = features_probability_boston["city_probability"]
        prob_canberra = features_probability_canberra["city_probability"]
        prob_geneva = features_probability_geneva["city_probability"]
        prob_ottowa = features_probability_ottawa["city_probability"]

        for word in x_i:
            if word in features_probability_boston: # All have the same most common words
                # Laplace smoothing
                prob_boston = prob_boston * ((features_probability_boston[word] + 1) / (features_probability_boston["city_count"] + 2))
                prob_canberra = prob_canberra * ((features_probability_canberra[word] + 1) / (features_probability_canberra["city_count"] + 2))
                prob_geneva = prob_geneva * ((features_probability_geneva[word] + 1) / (features_probability_geneva["city_count"] + 2))
                prob_ottowa = prob_ottowa * ((features_probability_ottawa[word] + 1) / (features_probability_ottawa["city_count"] + 2))
        
        probabilities = np.array([prob_boston, prob_canberra, prob_geneva, prob_ottowa])
        
        return np.argmax(probabilities)

    def accu_eval(self, x, y): # Issy
        # Validation
        # Returns Accuracy = 1 - Error

        num_correct_labels = 0

        # Get probabilities / train model
        features_probability_boston, features_probability_canberra, features_probability_geneva, features_probability_ottawa = self.calc_probability(x,y)

        # Predict
        for i in range(len(x)):
            predicted_label = self.predict(features_probability_boston, features_probability_canberra, features_probability_geneva, features_probability_ottawa, x[i])
            if predicted_label == y[i]:
                num_correct_labels += 1

        # Get accuracy
        accuracy = num_correct_labels/len(y)
        return accuracy

    def crossValidation(self, k): # Issy (PS: I think we are allowed to use the method from sklearn)
        kf = KFold(n_splits=k, shuffle=True, random_state=9)
        accuracies = []

        # Split into train and validation sets
        for train_indices, val_indices in kf.split(self.x_all):
            x_train = [self.x_all[i] for i in train_indices] # separate x into training subset
            x_val = [self.x_all[i] for i in val_indices] # separate x into validating subset

            y_train = [self.y_all[i] for i in train_indices] # separate y into training subset
            y_val = [self.y_all[i] for i in val_indices] # separate y into validating subset

            
            # For each set, get probabilities / train with training set
            features_probability_boston, features_probability_canberra, features_probability_geneva, features_probability_ottawa = self.calc_probability(x_train,np.array(y_train))

            # check with validation subset
            num_correct_labels = 0
            for i in range(len(y_val)):
                predicted_label = self.predict(features_probability_boston, features_probability_canberra, features_probability_geneva, features_probability_ottawa, x_val[i])
                if predicted_label == y_val[i]:
                    num_correct_labels += 1
        
            # calculate accuracy
            accuracy = num_correct_labels / len(y_val)
            accuracies.append(accuracy)

        self.folds_accuracy = accuracies
        self.avg_accuracy = np.mean(accuracies)
        return self.avg_accuracy


In [ ]:
# Naive Bayes set up
naiveBayes = NaiveBayes(X.to_numpy().flatten(), y.to_numpy(), feature_vectoriser=naiveBayes_uni)

accuracy = naiveBayes.crossValidation(28)
print(accuracy)

In [ ]:
# Compare with Naive Bayes model from sklearn
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

param_grid_NB_1 = {'alpha': np.arange(0.01, 1.11, 0.1)}

NB = hyperparamaterTunning(X_uni_arr[0], param_grid_NB_1 ,folds, MultinomialNB())

Fitting 30 folds for each of 11 candidates, totalling 330 fits
Mean Accuracy: 0.7166


<h2>Logistic Regression Models</h2>

In [ ]:
from sklearn.linear_model import LogisticRegression

param_grid_logModel_1 = [
    {"penalty":["elasticnet"],
     "l1_ratio": np.arange(0, 1.2, 0.2), # 0 is only l2 penalty, 1 is only l1 penalty
     "solver":["saga"],
     "max_iter": [1000]
     }]

logModel_tunned_1a = hyperparamaterTunning(X_uni, param_grid_logModel_1, folds, LogisticRegression(fit_intercept=True))
logModel_tunned_1b = hyperparamaterTunning(X_uni_bi, param_grid_logModel_1, folds, LogisticRegression(fit_intercept=True))


In [ ]:
from sklearn.linear_model import LogisticRegression

param_grid_logModel_2 = [
    {"penalty":["l2"],
     "solver":["sag","lbfgs","newton-cg"],
     "tol":[1e-4,1e-5],
     "max_iter": [1000,2000]
     }]
logModel_tunned_2a = hyperparamaterTunning(X_uni, param_grid_logModel_2, folds, LogisticRegression(fit_intercept=True))

logModel_tunned_2b = hyperparamaterTunning(X_uni_bi, param_grid_logModel_2, folds, LogisticRegression(fit_intercept=True))

Fitting 80 folds for each of 12 candidates, totalling 960 fits

Best Parameters: {'max_iter': 1000, 'penalty': 'l2', 'solver': 'lbfgs', 'tol': 0.0001}
Cross-validation Accuracies: [0.8333333333333334, 0.7222222222222222, 0.6111111111111112, 0.6111111111111112, 0.7777777777777778, 0.7222222222222222, 0.6111111111111112, 0.7222222222222222, 0.8333333333333334, 0.6111111111111112, 0.7777777777777778, 0.8333333333333334, 0.6666666666666666, 0.7222222222222222, 0.7777777777777778, 0.7222222222222222, 0.8333333333333334, 0.7777777777777778, 0.7777777777777778, 0.9444444444444444, 0.7222222222222222, 0.8333333333333334, 0.7777777777777778, 0.6666666666666666, 0.8333333333333334, 0.7222222222222222, 0.6111111111111112, 0.7222222222222222, 0.6111111111111112, 0.6111111111111112, 0.8888888888888888, 0.7777777777777778, 0.7222222222222222, 0.6111111111111112, 0.8888888888888888, 0.8888888888888888, 0.6111111111111112, 0.5555555555555556, 0.8888888888888888, 0.7777777777777778, 0.6470588235294118,

<h2>SVM Models</h2>

In [ ]:
from sklearn.svm import LinearSVC
param_grid_SVC_1 = [
    {"penalty":["l1","l2"],
     "C": np.arange(0.1,1.1,0.1).tolist(),
     "loss": ["squared_hinge"],
     "tol":[1e-4],
     "max_iter": [1000]
     }]

SVMModel_tunned_1a = hyperparamaterTunning(X_uni, param_grid_SVC_1, 14, LinearSVC(fit_intercept=True))

SVMModel_tunned_1b = hyperparamaterTunning(X_uni_bi, param_grid_SVC_1, 14, LinearSVC(fit_intercept=True))

In [ ]:
from sklearn.svm import LinearSVC
param_grid_SVC_2 = [
    {"penalty":["l2"],
     "C": np.arange(0.1,1.1,0.1).tolist(),
     "loss": ["hinge"],
     "tol":[1e-4],
     "max_iter": [1000]
     }]
# X_uni --> only discarded term that are smaller than 2

# MAX VALUE: 0.7415 -> Best Parameters:  {'C': 0.7000000000000001, 'loss': 'hinge', 'max_iter': 1000, 'penalty': 'l2', 'tol': 0.0001} using X1_uni_bi and 22 folds (lemmatized words, min_df = 1 and stop_words_custom)
meaninfullWords(X_uni)
SVMModel_tunned_2a = hyperparamaterTunning(X_uni, param_grid_SVC_2, 30, LinearSVC(fit_intercept=True)) # Best one so far
SVMModel_tunned_2b = hyperparamaterTunning(X_uni_bi, param_grid_SVC_2, 30, LinearSVC(fit_intercept=True)) # Best one so far

Fitting 30 folds for each of 10 candidates, totalling 300 fits
Mean Accuracy: 0.7314
Fitting 30 folds for each of 10 candidates, totalling 300 fits
Mean Accuracy: 0.7208


In [ ]:
from sklearn.svm import LinearSVC
param_grid_SVC_2 = [
    {"penalty":["l2"],
     "C": np.arange(0.1,1.1,0.1).tolist(),
     "loss": ["hinge"],
     "tol":[1e-4],
     "max_iter": [10000]
     }]
SVMModel_tunned_2 = []
for i in range(len(X_uni_arr)):
    print("Index:", i)
    for j in [22]:
        SVMModel_tunned_2.append(hyperparamaterTunning(X_uni_arr[i], param_grid_SVC_2, j, LinearSVC(fit_intercept=True)))
        print()



Index: 0
Fitting 22 folds for each of 10 candidates, totalling 220 fits


KeyboardInterrupt: 

In [419]:
param_grid_SVC_2 = [
    {"penalty":["l2"],
     "C": np.arange(0.1,1.1,0.1).tolist(),
     "loss": ["squared_hinge"],
     "tol":[1e-4],
     "max_iter": [10000]
     }]
X1,Y1,X2,Y2 = meaninfullWords(pp_df_train_1,y,meaningfull_words_list)

#hyperparamaterTunning(X_uni_primary, param_grid_SVC_2, 14, LinearSVC(fit_intercept=True),Y1)
print(len(Y1))
print(len(Y2))
print(len(X2))

tfidf_primary = TfidfVectorizer(vocabulary=meaningfull_words_list,sublinear_tf=True)
X_uni_primary = tfidf_primary.fit_transform(X1)
tfidf_secondary = TfidfVectorizer(ngram_range=(1, 1),sublinear_tf=True, stop_words=stop_words_custom)
X_uni_secondary = tfidf_secondary.fit_transform(pp_df_train_1)
print("Unigram", "(size:",str(len(tfidf_primary.get_feature_names_out()))+")")
print("Unigram", "(size:",str(len(tfidf_secondary.get_feature_names_out()))+")")


hyperparamaterTunning(X_uni_primary, param_grid_SVC_2, 30, LinearSVC(fit_intercept=True),Y=Y1)
hyperparamaterTunning(X_uni_secondary, param_grid_SVC_2, 30, LinearSVC(fit_intercept=True))
#for i in range(len(X_uni_bi_arr)):
#    print("Index:", i)
#    for j in [14, 30]:
#        hyperparamaterTunning(X_uni_bi_arr[i], param_grid_SVC_2, j, LinearSVC(fit_intercept=True))
#        print()

1237
163
163
Unigram (size: 470)
Unigram (size: 9125)
Fitting 30 folds for each of 10 candidates, totalling 300 fits

Best Parameters: {'C': 0.8, 'loss': 'squared_hinge', 'max_iter': 10000, 'penalty': 'l2', 'tol': 0.0001}
Cross-validation Accuracies: [0.5952380952380952, 0.5476190476190477, 0.6190476190476191, 0.6190476190476191, 0.5714285714285714, 0.6904761904761905, 0.6904761904761905, 0.6097560975609756, 0.6585365853658537, 0.5853658536585366, 0.6585365853658537, 0.7317073170731707, 0.7073170731707317, 0.6097560975609756, 0.6829268292682927, 0.7073170731707317, 0.6829268292682927, 0.5365853658536586, 0.6097560975609756, 0.7804878048780488, 0.5609756097560976, 0.6341463414634146, 0.5853658536585366, 0.6585365853658537, 0.5609756097560976, 0.6341463414634146, 0.6097560975609756, 0.6097560975609756, 0.6585365853658537, 0.6341463414634146]
Mean Accuracy: 0.6347
Fitting 30 folds for each of 10 candidates, totalling 300 fits

Best Parameters: {'C': 0.6, 'loss': 'squared_hinge', 'max_iter

GridSearchCV(cv=30, estimator=LinearSVC(),
             param_grid=[{'C': [0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6,
                                0.7000000000000001, 0.8, 0.9, 1.0],
                          'loss': ['squared_hinge'], 'max_iter': [10000],
                          'penalty': ['l2'], 'tol': [0.0001]}],
             verbose=1)

In [45]:
from sklearn.linear_model import SGDClassifier
# LOOK INTO THIS OR ELSE DELETE
param_grid_SDG = [
    {"penalty":["l1","l2"],
     "loss": ["hinge", "log_loss", "modified_huber", "squared_hinge", "perceptron", "squared_error", "huber", "epsilon_insensitive", "squared_epsilon_insensitive"],
     "tol":[1e-4],
     "max_iter": [1000]
     }]

param_grid_SDG = [
    {"penalty":["l2"],
     "alpha":[1e-3],
     "loss": ["hinge"],
     "tol":[1e-4],
     "max_iter": [1000]
     }]

# SDGModel = hyperparamaterTunning(X_uni_bi, param_grid_SDG, folds, SGDClassifier(fit_intercept=True),3)

<h2>Random Forest Model</h2>

In [ ]:
from sklearn.ensemble import RandomForestClassifier

param_grid_rf = [{
"criterion":["gini", "entropy", "log_loss"],
"max_features":["sqrt", "log2"],
"max_depth": [18] # Need to look into what values to use here
}]
rF = hyperparamaterTunning(X_uni, param_grid_rf, folds, RandomForestClassifier())

Fitting 80 folds for each of 6 candidates, totalling 480 fits

Best Parameters: {'criterion': 'gini', 'max_depth': 18, 'max_features': 'sqrt'}
Cross-validation Accuracies: [0.7777777777777778, 0.5, 0.6666666666666666, 0.5555555555555556, 0.5, 0.6666666666666666, 0.5555555555555556, 0.6666666666666666, 0.7222222222222222, 0.6111111111111112, 0.7222222222222222, 0.5, 0.5, 0.7222222222222222, 0.7222222222222222, 0.7777777777777778, 0.7222222222222222, 0.7222222222222222, 0.8333333333333334, 0.8888888888888888, 0.6666666666666666, 0.8888888888888888, 0.8333333333333334, 0.5555555555555556, 0.8333333333333334, 0.6111111111111112, 0.7222222222222222, 0.6666666666666666, 0.7222222222222222, 0.6666666666666666, 0.8333333333333334, 0.7222222222222222, 0.7222222222222222, 0.7222222222222222, 0.8888888888888888, 0.7777777777777778, 0.8333333333333334, 0.6666666666666666, 0.7777777777777778, 0.7222222222222222, 0.5882352941176471, 0.5294117647058824, 0.47058823529411764, 0.5882352941176471, 0.6470

<h3>Final Model<h3>

In [282]:
pp_df_test = preprocessing_data(df_test["body"],[])
X_test = tfidf_uni_arr[0].transform(pp_df_test)
#Test

y_pred = SVMModel_tunned_2[0].predict(X_test)


''' 
if (post has meaningfull Vocabulary):
    Run Model with meaningfull words as features only
else:
    Run Model with all words as features
'''

print(y_pred)
with open("output4.csv", mode='w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # Write a header (optional, if you want)
    writer.writerow(["id","subreddit"])
    map = ["Boston","Canberra","Geneva","Ottawa"]
    # Write the features from the array
    for i in range(len(y_pred)):
        output = y_pred[i]
        writer.writerow([i, map[output]])
    

[3 0 0 1 3 1 0 1 1 1 3 1 3 1 0 0 3 1 1 1 0 1 1 1 1 1 1 1 2 3 1 1 1 1 1 1 1
 1 1 1 1 1 1 2 1 2 1 0 1 0 1 0 1 2 3 3 0 1 1 1 0 1 1 1 1 1 0 1 1 1 3 1 0 1
 1 2 3 0 1 1 1 1 2 1 3 1 0 1 2 1 3 1 1 1 1 1 3 2 1 2 1 2 1 1 3 3 1 1 1 1 1
 2 2 1 1 1 1 2 0 1 3 1 2 1 1 0 1 0 1 1 3 0 2 1 2 2 3 1 1 1 1 1 3 1 3 1 1 3
 1 2 3 3 0 2 3 1 3 2 0 3 2 3 2 1 1 3 0 2 3 1 3 3 2 2 2 3 1 2 1 3 2 3 3 3 3
 2 3 3 3 1 3 3 3 0 0 3 3 2 3 3 3 3 3 3 2 0 3 3 3 0 3 3 1 0 3 3 3 3 3 2 3 3
 2 1 3 3 1 3 1 0 3 3 3 0 0 3 2 3 3 1 3 1 3 3 3 1 0 3 1 3 1 3 3 3 2 3 3 0 3
 3 3 3 3 3 0 3 3 3 1 3 3 1 3 3 3 1 2 3 0 1 3 3 2 3 1 2 2 3 3 3 1 2 3 3 0 0
 3 1 3 3 0 1 0 3 3 0 0 0 3 0 0 2 2 1 3 1 3 0 3 1 3 2 0 0 2 0 0 0 1 0 1 0 0
 3 0 0 0 0 0 1 0 0 3 3 0 2 0 0 0 0 0 0 3 0 0 0 0 0 0 0 3 0 0 0 3 0 0 2 0 0
 3 0 0 0 3 0 3 1 0 3 0 0 0 0 0 1 3 0 0 0 0 0 0 0 0 0 2 2 0 0 0 0 0 0 0 0 3
 0 1 0 0 2 0 2 0 0 0 0 2 0 0 0 0 0 3 3 0 0 0 3 0 0 3 2 0 3 0 0 3 3 0 3 0 2
 0 0 0 2 0 0 2 0 3 0 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 0 2 2 2 2 2 2 2 2 2 3 2
 2 2 2 3 2 2 0 2 2 2 1 2 